In [5]:
from z3 import Optimize, Real, sat
from fractions import Fraction

def calc_value_added_tax(output_tax_val: float, input_tax_val: float, free_vars: list = None):
    """
    計算加值型營業稅：最終稅額 = 銷項稅額 - 進項稅額
    同時利用 Optimize.minimize() 求解，並印出優化過程（不再迭代）。
    
    參數：
      output_tax_val: 銷項稅額輸入值
      input_tax_val:  進項稅額輸入值
      free_vars:      指定哪些變數名稱為自由變數 (例如 ["input_tax_amount"])
      
    回傳：
      tuple: (最終最佳稅額, 最終參數狀態字典, 變數變動訊息字典)
    """
    if free_vars is None:
        free_vars = []
    
    # 使用 Optimize 求解器
    opt = Optimize()

    # 定義變數
    output_tax_amount = Real('output_tax_amount')
    input_tax_amount  = Real('input_tax_amount')
    final_tax_amount  = Real('final_tax_amount')

    # 建立「輸入值 -> Z3 變數」對應字典，格式：(Z3 變數, 原始數值, 約束條件列表)
    params = {
        "output_tax_amount": (output_tax_amount, output_tax_val, []),
        "input_tax_amount":  (input_tax_amount,  input_tax_val, [])
    }
    
    # 若不在 free_vars 中則固定參數；在 free_vars 中僅加入其他約束
    for name, (var, value, cons_list) in params.items():
        if free_vars and (name in free_vars):
            for cons in cons_list:
                opt.add(cons(var))
        else:
            opt.add(var == value)
            for cons in cons_list:
                opt.add(cons(var))
    
    # 計算公式：最終稅額 = 銷項稅額 - 進項稅額
    opt.add(final_tax_amount == output_tax_amount - input_tax_amount)
    
    # --- 先利用 push/pop 固定 free_vars 取得初始稅額（基準解） ---
    if free_vars:
        opt.push()
        for name in free_vars:
            var, orig_val, _ = params[name]
            opt.add(var == orig_val)
        if opt.check() == sat:
            fixed_model = opt.model()
            fixed_tax = fixed_model[final_tax_amount].as_long()
        else:
            fixed_tax = None
        opt.pop()
    else:
        if opt.check() == sat:
            fixed_tax = opt.model()[final_tax_amount].as_long()
        else:
            fixed_tax = None

    print("初始計算（固定 free_vars）稅額：{}".format(fixed_tax))
    print("--------------------------------------------------\n")
    print("自由變數有：{}\n".format(free_vars))
    print("=== 優化過程開始 ===\n")
    print("呼叫 optimize.minimize() 求解中...\n")
    
    # --- 使用 Optimize 的 minimize 方法求最佳解 ---
    obj = opt.minimize(final_tax_amount)
    if opt.check() == sat:
        m = opt.model()
        best_tax = m[final_tax_amount].as_long()
    else:
        raise ValueError("Optimize 無法找到滿足所有約束的解！")
    
    print("=== 優化結束 ===")
    print("最終最佳稅額 = {}\n".format(best_tax))
    
    # 建立最終參數字典與差異訊息（對比原始值）
    final_params = {}
    differences = {}
    for name, (var, orig_val, _) in params.items():
        try:
            opt_val = m.eval(var).as_long()
        except Exception:
            opt_val = None
        final_params[name] = {
            "value": opt_val,
            "type": "free" if free_vars and (name in free_vars) else "fixed"
        }
        if opt_val is not None and opt_val != orig_val:
            differences[name] = {
                "original": orig_val,
                "optimized": opt_val,
                "difference": opt_val - orig_val
            }
    return best_tax, final_params, differences


def calc_non_value_added_tax(sales_val: float, category: int, free_vars: list = None):
    """
    根據非加值型營業稅邏輯計算應納稅額：最終稅額 = 銷售額 * 稅率
    稅率根據 category 從 TAX_RATE_MAP 取得。
    此函式利用 Optimize.minimize() 求得最優解，並印出求解過程（不使用迭代）。
    
    參數：
      sales_val: 銷售額
      category:  類別代號 (1~8)
      free_vars: 指定哪些變數名稱為自由變數 (例如 ["sales_amount"])
      
    回傳：
      tuple: (最終稅額, 最終參數狀態字典, 變數變動訊息字典)
    """
    if free_vars is None:
        free_vars = []
    
    TAX_RATE_MAP = {
        1: 0.01,   # 1%
        2: 0.01,   # 1%
        3: 0.02,   # 2%
        4: 0.05,   # 5%
        5: 0.05,   # 5%
        6: 0.15,   # 15%
        7: 0.25,   # 25%
        8: 0.001   # 0.1%
    }
    if category not in TAX_RATE_MAP:
        raise ValueError(f"類別代號 {category} 不在稅率對應表中，請確認輸入是否正確。")
    tax_rate_val = TAX_RATE_MAP[category]
    
    # 使用 Optimize 求解器
    opt = Optimize()

    # 定義變數
    sales_amount = Real('sales_amount')
    tax_rate = Real('tax_rate')
    final_tax_amount = Real('final_tax_amount')

    # 參數對應字典
    params = {
        "sales_amount": (sales_amount, sales_val, []),
        "tax_rate":     (tax_rate, tax_rate_val, [])
    }
    
    for name, (var, value, cons_list) in params.items():
        if free_vars and (name in free_vars):
            for cons in cons_list:
                opt.add(cons(var))
        else:
            opt.add(var == value)
            for cons in cons_list:
                opt.add(cons(var))
    
    # 計算公式：最終稅額 = 銷售額 * 稅率
    opt.add(final_tax_amount == sales_amount * tax_rate)
    
    # --- 先固定 free_vars 取得初始解 ---
    if free_vars:
        opt.push()
        for name in free_vars:
            var, orig_val, _ = params[name]
            opt.add(var == orig_val)
        if opt.check() == sat:
            fixed_model = opt.model()
            fixed_tax = fixed_model[final_tax_amount].as_long()
        else:
            fixed_tax = None
        opt.pop()
    else:
        if opt.check() == sat:
            fixed_tax = opt.model()[final_tax_amount].as_long()
        else:
            fixed_tax = None

    print("初始計算（固定 free_vars）稅額：{}".format(fixed_tax))
    print("--------------------------------------------------\n")
    print("自由變數有：{}\n".format(free_vars))
    print("=== 優化過程開始 ===\n")
    print("呼叫 optimize.minimize() 求解中...\n")
    
    # --- 使用 Optimize 的 minimize 方法求最佳解 ---
    obj = opt.minimize(final_tax_amount)
    if opt.check() == sat:
        m = opt.model()
        best_tax = m[final_tax_amount].as_long()
    else:
        raise ValueError("Optimize 無法找到滿足所有約束的解！")
    
    print("=== 優化結束 ===")
    print("最終最佳稅額 = {}\n".format(best_tax))
    
    final_params = {}
    differences = {}
    for name, (var, orig_val, _) in params.items():
        try:
            opt_val = m.eval(var).as_long()
        except Exception:
            opt_val = None
        final_params[name] = {
            "value": opt_val,
            "type": "free" if free_vars and (name in free_vars) else "fixed"
        }
        if opt_val is not None and opt_val != orig_val:
            differences[name] = {
                "original": orig_val,
                "optimized": opt_val,
                "difference": opt_val - orig_val
            }
    return best_tax, final_params, differences


if __name__ == "__main__":
    # ============ 範例 1：加值型營業稅 =============
    # 例如：銷項稅額 = 750, 進項稅額 = 500, 應納稅額 = 750 - 500 = 250
    # 若指定 free_vars=["input_tax_amount"]，則 input_tax_amount 會在最佳化中自由調整
    best_vat, final_params_vat, diffs_vat = calc_value_added_tax(
        output_tax_val=750, 
        input_tax_val=500, 
        free_vars=["input_tax_amount"]
    )
    print("[加值型營業稅] 最佳稅額 = {}".format(best_vat))
    print("最終參數狀態 = {}".format(final_params_vat))
    print("變數變動訊息 = {}\n".format(diffs_vat))
    
    # ============ 範例 2：非加值型營業稅 =============
    # 以第 1 類（稅率 1%）為例，銷售額 = 100000
    best_nv, final_params_nv, diffs_nv = calc_non_value_added_tax(
        sales_val=100000, 
        category=1, 
        free_vars=["sales_amount"]
    )
    print("[非加值型營業稅] 最佳稅額 = {}".format(best_nv))
    print("最終參數狀態 = {}".format(final_params_nv))
    print("變數變動訊息 = {}".format(diffs_nv))


初始計算（固定 free_vars）稅額：250
--------------------------------------------------

自由變數有：['input_tax_amount']

=== 優化過程開始 ===

呼叫 optimize.minimize() 求解中...

=== 優化結束 ===
最終最佳稅額 = 0

[加值型營業稅] 最佳稅額 = 0
最終參數狀態 = {'output_tax_amount': {'value': 750, 'type': 'fixed'}, 'input_tax_amount': {'value': 750, 'type': 'free'}}
變數變動訊息 = {'input_tax_amount': {'original': 500, 'optimized': 750, 'difference': 250}}

初始計算（固定 free_vars）稅額：1000
--------------------------------------------------

自由變數有：['sales_amount']

=== 優化過程開始 ===

呼叫 optimize.minimize() 求解中...

=== 優化結束 ===
最終最佳稅額 = 0

[非加值型營業稅] 最佳稅額 = 0
最終參數狀態 = {'sales_amount': {'value': 0, 'type': 'free'}, 'tax_rate': {'value': None, 'type': 'fixed'}}
變數變動訊息 = {'sales_amount': {'original': 100000, 'optimized': 0, 'difference': -100000}}
